In [1]:
!nvidia-smi
!pip -q install -U pip setuptools wheel
# Full AutoGluon includes tabular + multimodal
!pip -q install -U "autogluon>=1.1.0"


/bin/bash: line 1: nvidia-smi: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This be

In [2]:
import pandas as pd
from autogluon.tabular import TabularPredictor

train_path = "/content/sample_data/california_housing_train.csv"
test_path  = "/content/sample_data/california_housing_test.csv"

train_df = pd.read_csv(train_path)
test_df  = pd.read_csv(test_path)

label = "median_house_value"
print("Train shape:", train_df.shape, " Test shape:", test_df.shape)
train_df.head()


Train shape: (17000, 9)  Test shape: (3000, 9)


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


In [3]:
# Train (fast preset for quick completion)
predictor = TabularPredictor(label=label, eval_metric="r2", path="/content/ag_california_tabular").fit(
    train_df,
    presets="medium_quality_faster_train",  # good speed/quality tradeoff
    time_limit=300                           # 5 minutes cap (adjust if you want)
)


Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Memory Avail:       10.97 GB / 12.67 GB (86.6%)
Disk Space Avail:   61.59 GB / 107.72 GB (57.2%)
Presets specified: ['medium_quality_faster_train']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "/content/ag_california_tabular"
Train Data Rows:    17000
Train Data Columns: 8
Label Column:       median_house_value
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (500001.0, 14999.0, 207300.91235, 115983.76439)
	If 'regression' is no

[1000]	valid_set's l2: 2.27678e+09	valid_set's r2: 0.829396
[2000]	valid_set's l2: 2.16813e+09	valid_set's r2: 0.837537
[3000]	valid_set's l2: 2.13862e+09	valid_set's r2: 0.839748
[4000]	valid_set's l2: 2.1153e+09	valid_set's r2: 0.841496
[5000]	valid_set's l2: 2.11423e+09	valid_set's r2: 0.841576
[6000]	valid_set's l2: 2.11028e+09	valid_set's r2: 0.841872
[7000]	valid_set's l2: 2.11215e+09	valid_set's r2: 0.841732


	0.8422	 = Validation score   (r2)
	37.43s	 = Training   runtime
	3.94s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 256.84s of the 256.84s of remaining time.
	Fitting with cpus=1, gpus=0, mem=0.0/10.7 GB


[1000]	valid_set's l2: 1.89868e+09	valid_set's r2: 0.857728


	0.8583	 = Validation score   (r2)
	3.92s	 = Training   runtime
	0.16s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 252.63s of the 252.62s of remaining time.
	Fitting with cpus=2, gpus=0, mem=0.0/10.7 GB
	0.8348	 = Validation score   (r2)
	45.2s	 = Training   runtime
	0.31s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 200.24s of the 200.23s of remaining time.
	Fitting with cpus=1, gpus=0
	0.8585	 = Validation score   (r2)
	80.39s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ... Training model for up to 119.80s of the 119.80s of remaining time.
	Fitting with cpus=2, gpus=0, mem=0.0/10.4 GB
	0.7981	 = Validation score   (r2)
	12.22s	 = Training   runtime
	0.3s	 = Validation runtime
Fitting model: NeuralNetFastAI ... Training model for up to 100.47s of the 100.47s of remaining time.
	Fitting with cpus=1, gpus=0, mem=0.0/10.4 GB
	0.8128	 = Validation score   (r2)
	26.52s	 = Training   runt

In [4]:
# Evaluate on a small validation split (holdout from the training data)
leaderboard = predictor.leaderboard(train_df, silent=True)
leaderboard


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE,0.961974,0.834826,r2,2.217884,0.305984,45.200148,2.217884,0.305984,45.200148,1,True,3
1,ExtraTreesMSE,0.953941,0.798127,r2,1.506224,0.299927,12.221332,1.506224,0.299927,12.221332,1,True,5
2,CatBoost,0.945728,0.858477,r2,0.097768,0.011100,80.388021,0.097768,0.011100,80.388021,1,True,4
3,XGBoost,0.944851,0.848040,r2,0.387777,0.043249,3.248294,0.387777,0.043249,3.248294,1,True,7
4,WeightedEnsemble_L2,0.942700,0.863968,r2,4.484324,0.506996,200.274642,0.004630,0.002136,0.198913,2,True,9
5,LightGBMXT,0.934617,0.842152,r2,37.162615,3.936930,37.429081,37.162615,3.936930,37.429081,1,True,1
6,LightGBM,0.931055,0.858260,r2,2.063370,0.157573,3.922986,2.063370,0.157573,3.922986,1,True,2
7,NeuralNetTorch,0.830319,0.814966,r2,0.100671,0.030204,70.564575,0.100671,0.030204,70.564575,1,True,8
8,NeuralNetFastAI,0.816390,0.812773,r2,0.279683,0.034583,26.517126,0.279683,0.034583,26.517126,1,True,6


In [5]:
import numpy as np

# Make an Id column (since the sample_test has no explicit Id)
test_with_id = test_df.copy()
test_with_id["Id"] = range(len(test_with_id))

preds = predictor.predict(test_with_id)
submission = test_with_id[["Id"]].copy()
submission[label] = preds.astype(float)

out_path = "/content/submission_tabular.csv"
submission.to_csv(out_path, index=False)
out_path


'/content/submission_tabular.csv'

In [6]:
from google.colab import files
files.download("/content/submission_tabular.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>